In [1]:
# import packages
import pandas as pd, numpy as np
import json
# import files
import data as dt

1. Replicar 100 bps? -> **10 Basis points: dejar como parametro del backtest**
2. Determinar origin vs destination exchange -> **Kraken es origin, Bitfinex es destination**
3. Como replicar los niveles del origin en el destination (volumen)
4. Validar el método para determinar maker/taker fee: iterativo vs estático
5. Orden de llenado por tiempo de entrada.
6. Evolución de Orderbooks tipo bootstrap.

In [2]:
# load data
ob_krak, ob_bit = dt.read_jsonOB(file_name = 'orderbooks_05jul21.json')


In [3]:
type(ob_krak)

dict

# Alineación de timestamps

In [3]:
list(ob_krak.keys())[0], list(ob_bit.keys())[0], list(ob_bit.keys())[1], list(ob_bit.keys())[2], list(ob_bit.keys())[3]

('2021-07-05T13:06:49.495Z',
 '2021-07-05T13:06:46.571Z',
 '2021-07-05T13:06:47.918Z',
 '2021-07-05T13:06:49.414Z',
 '2021-07-05T13:06:51.077Z')

In [4]:
# filtro  para que destination sea despues de origin
aux = sum(pd.to_datetime(pd.Series((ob_bit.keys()))) < pd.to_datetime(pd.Series(ob_krak.keys()))[0])
[ob_bit.pop(key) for key in list(ob_bit.keys())[:aux]]

[     bid_size      bid      ask  ask_size
 0    0.000400  28270.0  28275.0  0.025405
 1    0.009787  28269.0  28276.0  0.516810
 2    0.008168  28268.0  28277.0  0.005044
 3    0.995787  28266.0  28278.0  0.377374
 4    1.038704  28265.0  28280.0  1.179715
 5    1.169713  28264.0  28281.0  0.287236
 6    0.133730  28263.0  28282.0  0.233462
 7    0.245015  28260.0  28283.0  1.072689
 8    0.947100  28257.0  28284.0  0.343960
 9    0.482000  28256.0  28285.0  0.107199
 10  16.088585  28255.0  28286.0  0.704387
 11   0.034721  28254.0  28287.0  0.755217
 12   1.810782  28253.0  28288.0  2.937022
 13   0.655256  28251.0  28289.0  0.623758
 14   0.069447  28250.0  28290.0  4.056746
 15   0.104400  28249.0  28291.0  0.075260
 16   0.708510  28248.0  28292.0  1.957100
 17   1.447259  28247.0  28293.0  1.204343
 18   1.489984  28246.0  28294.0  2.923809
 19   0.010000  28245.0  28295.0  6.041307
 20   2.809104  28244.0  28296.0  3.424052
 21   0.756619  28243.0  28297.0  0.005064
 22   0.697

# Origin exchange (Kraken)

In [6]:
# OB 1
df_krak = ob_krak[list(ob_krak.keys())[0]]
# midprice
mid_krak = (df_krak['bid'][0]+df_krak['ask'][0])/2
display(df_krak.head(3))
mid_krak

,bid_size,bid,ask,ask_size
0,0.880,28298.3,28298.4,3.352
1,0.350,28293.9,28300.0,0.020
2,0.377,28293.5,28305.4,0.074


28298.35

In [7]:
upper_krak = mid_krak*1.001
lower_krak = mid_krak*0.999
(lower_krak, upper_krak)

(28270.051649999998, 28326.648349999996)

In [8]:
bids_krak = df_krak[df_krak['bid']>lower_krak][['bid','bid_size']]
asks_krak = df_krak[df_krak['ask']<upper_krak][['ask','ask_size']]

In [9]:
[d.shape for d in [df_krak, bids_krak, asks_krak]]

[(100, 4), (30, 2), (23, 2)]

In [10]:
levels = pd.concat([bids_krak, asks_krak], axis=1)

In [11]:
levels

,bid,bid_size,ask,ask_size
0,28298.3,0.880,28298.4,3.352
1,28293.9,0.350,28300.0,0.020
2,28293.5,0.377,28305.4,0.074
3,28292.2,0.011,28305.5,0.123
4,28292.0,1.221,28307.3,0.204
5,28291.2,0.745,28308.2,0.005
6,28291.0,0.396,28310.0,0.004
7,28290.9,0.340,28311.8,0.745
8,28290.8,0.006,28314.9,0.250
9,28290.0,0.003,28315.2,5.000


# Destination exchange (Bitfinex)

In [12]:
# OB 1
df_bit = ob_bit[list(ob_bit.keys())[0]]
# midprice
mid_bit = (df_bit['bid'][0]+df_bit['ask'][0])/2
display(df_bit.head(3))
mid_bit

,bid_size,bid,ask,ask_size
0,14.605659,28275.0,28278.0,0.011063
1,0.005124,28273.0,28279.0,0.006000
2,0.010158,28272.0,28280.0,0.100000


28276.5

# Preliminary approach using loops

In [13]:
# destination exchange
bit_ask = df_bit[['ask','ask_size']]
bit_bid = df_bit[['bid_size','bid']]

bit_ask['ask_added_vol'] = np.zeros(bit_ask.shape[0])
bit_bid['bid_added_vol'] = np.zeros(bit_bid.shape[0])

# origin exchnage
krak_bid = df_krak[['bid_size','bid']]
krak_ask = df_krak[['ask','ask_size']]

<ipython-input-13-3c8ae2bf6895>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bit_ask['ask_added_vol'] = np.zeros(bit_ask.shape[0])
<ipython-input-13-3c8ae2bf6895>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bit_bid['bid_added_vol'] = np.zeros(bit_bid.shape[0])


In [14]:
bid_levels = levels[['bid_size','bid']]
bid_levels['bid_flag'] = ['bid']*bid_levels.shape[0]
bid_levels.rename(columns={'bid_size':'size',
                          'bid':'price',
                          'bid_flag':'type'}, inplace=True)

ask_levels = levels[['ask_size','ask']]
ask_levels['ask_flag'] = ['ask']*ask_levels.shape[0]
ask_levels.rename(columns={'ask_size':'size',
                          'ask':'price',
                          'ask_flag':'type'}, inplace=True)

levels_added = bid_levels.append(ask_levels, ignore_index=True).dropna()
np.random.seed(123)
levels_added['transaction_time'] = np.random.uniform(1250, size=levels_added.shape[0])
levels_added.sort_values(by='transaction_time', inplace=True, ignore_index=True)

<ipython-input-14-e9da3bbfbd45>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bid_levels['bid_flag'] = ['bid']*bid_levels.shape[0]
C:\Users\if724\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-14-e9da3bbfbd45>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

In [15]:
levels_added

,size,price,type,transaction_time
0,0.300,28320.9,ask,19.035828
1,0.396,28291.0,bid,25.025516
2,0.250,28314.9,ask,70.744137
3,0.745,28311.8,ask,134.156935
4,1.729,28317.0,ask,167.979862
5,0.300,28280.2,bid,189.059689
6,1.089,28322.7,ask,217.900340
7,0.300,28283.6,bid,328.243738
8,1.491,28288.7,bid,339.416915
9,1.491,28279.4,bid,345.155299


### Notas de asesoría

* Añadir latencia de código para simular el tiempo transcurrido en la operación (VARIABLE)
* Si el tiempo total de latencia para llenar todas las ordenes es mayor a el tiempo entre OB update entonces algunas ordenes no se llenan
* El hedge en el origin no es posible realizarlo al instante, existe latencia en procesamiento de transacción.
* Spread en origin se puede reducir introduciendo sólo una fracción del volumen del origin en el destination.
* Replicar niveles en del origin con el inventario en ambos exchanges como restricción
* SLA: 10 bps a partir del Origin ToB y 'x' cantidad de volumen mínimo
* Insights
* **Especificar supuestos**

In [11]:
# storing dictionary
ob_xemm = {}

# Initial parameters
bp = 10 
# Origin OB
df_krak = ob_krak[list(ob_krak.keys())[0]]
krak_bid = df_krak[['bid_size','bid']]
krak_ask = df_krak[['ask','ask_size']]
mid_krak = (df_krak['bid'][0]+df_krak['ask'][0])/2


# Destination OB
df_bit = ob_bit[list(ob_bit.keys())[0]]
bit_ask = df_bit[['ask','ask_size']]
bit_bid = df_bit[['bid_size','bid']]
bit_ask['ask_added_vol'] = np.zeros(bit_ask.shape[0])
bit_bid['bid_added_vol'] = np.zeros(bit_bid.shape[0])

for t in range(4): #len(list(ob_bit.keys()))-1
    
    upper_krak = mid_krak*1 + bp/10000
    lower_krak = mid_krak*1 - bp/10000
    
    bids_krak = df_krak[df_krak['bid']>lower_krak][['bid','bid_size']]
    asks_krak = df_krak[df_krak['ask']<upper_krak][['ask','ask_size']]
    levels = pd.concat([bids_krak, asks_krak], axis=1)
    
    # replicate origin levels on destination
    bid_levels = levels[['bid_size','bid']]
    bid_levels['bid_flag'] = ['bid']*bid_levels.shape[0]
    bid_levels.rename(columns={'bid_size':'size',
                              'bid':'price',
                              'bid_flag':'type'}, inplace=True)
    
    ask_levels = levels[['ask_size','ask']]
    ask_levels['ask_flag'] = ['ask']*ask_levels.shape[0]
    ask_levels.rename(columns={'ask_size':'size',
                              'ask':'price',
                              'ask_flag':'type'}, inplace=True)
    
    levels_added = bid_levels.append(ask_levels, ignore_index=True).dropna()
    np.random.seed(123)
    
    # sort queue by transaction time
    levels_added['transaction_time'] = np.random.uniform(1250, size=levels_added.shape[0])
    levels_added.sort_values(by='transaction_time', inplace=True, ignore_index=True)

    for i in range(levels_added.shape[0]): # loop to add levels unto destination

        to_fill = levels_added.iloc[0,:]

        if to_fill['type']=='bid':
            # identify order's fee structure
            fee_type = 'taker' if to_fill['price'] >= bit_ask.iloc[0]['ask'] else 'maker'


            if fee_type=='taker':

                bit_ask['accum_size'] = bit_ask['ask_size'].cumsum()


                vol_to_fill=to_fill['size']

                bit_ask['remaining_vol'] = vol_to_fill - bit_ask['accum_size']
                to_drop = bit_ask[bit_ask['remaining_vol']>0]

                bit_ask.drop(to_drop.index, inplace=True)
                bit_ask.reset_index(drop=True, inplace=True)

                new_vol = -bit_ask.iloc[0,-1]  # remaing volume on surviving level

                bit_ask.iloc[0,1] = new_vol
                bit_ask.drop(['accum_size', 'remaining_vol'], axis=1, inplace=True) 
                bit_ask.reset_index(drop=True, inplace=True)

                levels_added.drop(0, inplace=True)
                levels_added.reset_index(drop=True, inplace=True)

                # Hedge transaction in orgin exchange (Kraken)

                krak_bid['accum_size'] = krak_bid['bid_size'].cumsum()
                krak_bid['remaining_vol'] = vol_to_fill - krak_bid['accum_size']
                to_drop = krak_bid[krak_bid['remaining_vol']>0]

                krak_bid.drop(to_drop.index, inplace=True)
                krak_bid.reset_index(drop=True, inplace=True)

                new_vol = -krak_bid.iloc[0,-1] # remaing volume on surviving level

                krak_bid.iloc[0,0] = new_vol
                krak_bid.drop(['accum_size','remaining_vol'], axis=1, inplace=True)
                krak_bid.reset_index(drop=True, inplace=True)

            else:

                bit_bid.rename(columns={'bid_size':'size','bid':'price'}, inplace=True)

                to_fill = pd.DataFrame(to_fill).transpose()
                to_fill['bid_added_vol'] = to_fill['size'].values

                bit_bid = bit_bid.merge(to_fill[['size','price','bid_added_vol']], 
                                        on=['size','price','bid_added_vol'], how='outer'
                                       ).sort_values(by='price', ascending=False, ignore_index=True) 

                bit_bid.rename(columns={'size':'bid_size','price':'bid'}, inplace=True)
                bit_bid = bit_bid.groupby('bid', as_index=False, sort=False).sum()
                bit_bid = bit_bid.reindex(columns=['bid_added_vol', 'bid_size', 'bid'])


                levels_added.drop(0, inplace=True)
                levels_added.reset_index(drop=True, inplace=True)

        else:

            # identify order's fee structure
            fee_type = 'taker' if to_fill['price'] < bit_bid.iloc[0]['bid'] else 'maker'            

            if fee_type == 'taker':

                bit_bid['accum_size'] = bit_bid['bid_size'].cumsum()


                vol_to_fill = to_fill['size']

                bit_bid['remaining_vol'] = vol_to_fill - bit_bid['accum_size']
                to_drop = bit_bid[bit_bid['remaining_vol']>0]

                bit_bid.drop(to_drop.index, inplace=True)
                bit_bid.reset_index(drop=True, inplace=True)

                new_vol = -bit_bid.iloc[0,-1] # remaing volume on surviving level

                bit_bid.iloc[0,0] = new_vol
                bit_bid.drop(['accum_size','remaining_vol'], axis=1, inplace=True)
                bit_bid.reset_index(drop=True, inplace=True)

                levels_added.drop(0, inplace=True)
                levels_added.reset_index(drop=True, inplace=True)

                # Hedge transaction in origin market (Kraken)
                krak_ask['accum_size'] = krak_ask['ask_size'].cumsum()
                krak_ask['remaining_vol'] = vol_to_fill - krak_ask['accum_size']
                to_drop = krak_ask[krak_ask['remaining_vol']>0]

                krak_ask.drop(to_drop.index, inplace=True)
                krak_ask.reset_index(drop=True, inplace=True)

                new_vol = -krak_ask.iloc[0,-1] # remaing volume on surviving level

                krak_ask.iloc[0,1] = new_vol
                krak_ask.drop(['accum_size','remaining_vol'], axis=1, inplace=True)
                krak_ask.reset_index(drop=True, inplace=True)


            else:
                bit_ask.rename(columns={'ask_size':'size','ask':'price'}, inplace=True)

                to_fill = pd.DataFrame(to_fill).transpose()
                to_fill['ask_added_vol'] = to_fill['size'].values

                bit_ask = bit_ask.merge(to_fill[['size','price','ask_added_vol']], 
                                        on=['size','price','ask_added_vol'], how='outer'
                                       ).sort_values(by='price', ascending=True, ignore_index=True)               
                bit_ask.rename(columns={'size':'ask_size','price':'ask'}, inplace=True)
                bit_ask = bit_ask.groupby('ask', as_index=False, sort=False).sum()
                bit_ask = bit_ask.reindex(columns=['ask', 'ask_size', 'ask_added_vol'])


                levels_added.drop(0, inplace=True)
                levels_added.reset_index(drop=True, inplace=True)


    # Destination after adding origin's levels
    bit_ob = pd.concat([bit_bid, bit_ask], axis=1)
    next_ob_bit = ob_bit[list(ob_bit.keys())[t+1]]
    new_tob = (next_ob_bit.loc['0','bid'], next_ob_bit.loc['0','ask'])

    # levels to drop after comparison with next destination TOB
    bids_to_drop = bit_ob[bit_ob['bid']>new_tob[0]]
    asks_to_drop = bit_ob[bit_ob['ask']<new_tob[1]]

    bit_ob.drop(bids_to_drop.index, inplace=True)
    bit_ob.reset_index(drop=True, inplace=True)
    bit_ob.drop(asks_to_drop.index, inplace=True)
    bit_ob.reset_index(drop=True, inplace=True)    

    # Destination XEMM output OrderBook
    bit_bid_new_tob = bit_ob[['bid_added_vol','bid_size','bid']]
    bit_ask_new_tob = bit_ob[['ask_added_vol','ask_size','ask']]

    # Destination's next OrderBook
    next_ob_bit_bids = next_ob_bit[['bid_size','bid']]
    next_ob_bit_asks = next_ob_bit[['ask','ask_size']]
    next_ob_bit_bids['bid_added_vol'] = np.zeros(next_ob_bit_bids.shape[0]) 
    next_ob_bit_bids['ask_added_vol'] = np.zeros(next_ob_bit_asks.shape[0]) 


    # Origin's next OrderBook
    next_ob_krak = ob_krak[list(ob_krak.keys())[t+1]] 
    next_ob_krak_bids = next_ob_krak[['bid_size','bid']]
    next_ob_krak_asks = next_ob_krak[['ask_size','ask']]
    
        
    # generate new OrderBook based on next OB data (modify depth of output OB 'bit_ob')
    
    # Bids
    
    # data prep
    new_vols_bid = pd.merge(bit_bid_new_tob, next_ob_bit_bids, on='bid', how='outer').fillna(0)
    new_vols_bid.sort_values(by='bid', ascending=False, ignore_index=True, inplace=True)
    new_vols_bid.rename(columns={'bid_size_x':'current_vol','bid_size_y':'next_vol'}, inplace=True)
    
    # procedure
    new_vols_bid['original_vol'] = new_vols_bid['current_vol'] - new_vols_bid['bid_added_vol']
    
    # scenario a: existing level w/o volume in next ob
    new_vols_bid['new_vol'] = np.zeros(new_vols_bid.shape[0]) 
    
    # scenario b: added level with volume in next ob
    indx_b = new_vols_bid[(new_vols_bid['original_vol']==0) & (new_vols_bid['next_vol']!=0)].index
    new_vols_bid.loc[indx_b,'new_vol'] = new_vols_bid.loc[indx_b, 'current_vol'] + new_vols_bid.loc[indx_b,'next_vol'] 
    
    # scenario c: added level w/o volume in next ob
    indx_c = new_vols_bid[(new_vols_bid['original_vol']==0) & (new_vols_bid['next_vol']==0)].index
    new_vols_bid.loc[indx_c,'new_vol'] = new_vols_bid.loc[indx_c, 'bid_added_vol']
        
    # scenario d: existing level with volume in next ob
    
    indx_d = new_vols_bid[(new_vols_bid['original_vol']!=0) & (new_vols_bid['next_vol']!=0)].index
    new_vols_bid.loc[indx_d,'new_vol'] = new_vols_bid.loc[indx_d, 'current_vol'] + (new_vols_bid.loc[indx_d, 'next_vol'] 
                                                                                    - new_vols_bid.loc[indx_d, 'original_vol'])
    
    # identify filled orders
    
    # output
    df_bid = new_vols_bid[['bid_added_vol','new_vol', 'bid']]
    df_bid.rename(columns={'new_vol':'bid_size'}, inplace=True)
    
    # Asks
    
    # data prep
    new_vols_ask = pd.merge(bit_ask_new_tob, next_ob_bit_asks, on='ask', how='outer').fillna(0)
    new_vols_ask.sort_values(by='ask', ignore_index=True, inplace=True)
    new_vols_ask.rename(columns={'ask_size_x':'current_vol','ask_size_y':'next_vol'}, inplace=True)
        
    # procedure
    new_vols_ask['original_vol'] = new_vols_ask['current_vol'] - new_vols_ask['ask_added_vol']
    new_vols_ask.drop(new_vols_ask[new_vols_ask['ask']==0].index, inplace=True) # drop levels whose price is 0
    new_vols_ask.reset_index(drop=True, inplace=True)
    
    # scenario a: existing level w/o volume in next ob
    new_vols_ask['new_vol'] = np.zeros(new_vols_ask.shape[0]) 
    
    # scenario b: added level with volume in next ob
    indx_b = new_vols_ask[(new_vols_ask['original_vol']==0) & (new_vols_ask['next_vol']!=0)].index
    new_vols_ask.loc[indx_b,'new_vol'] = new_vols_ask.loc[indx_b, 'current_vol'] + new_vols_ask.loc[indx_b,'next_vol'] 
    
    # scenario c: added level w/o volume in next ob
    indx_c = new_vols_ask[(new_vols_ask['original_vol']==0) & (new_vols_ask['next_vol']==0)].index
    new_vols_ask.loc[indx_c,'new_vol'] = new_vols_ask.loc[indx_c, 'ask_added_vol']
    
    # scenario d: existing level with volume in next ob
    indx_d = new_vols_ask[(new_vols_ask['original_vol']!=0) & (new_vols_ask['next_vol']!=0)].index
    new_vols_ask.loc[indx_d,'new_vol'] = new_vols_ask.loc[indx_d, 'current_vol'] + (new_vols_ask.loc[indx_d, 'next_vol'] 
                                                                                    - new_vols_ask.loc[indx_d, 'original_vol'])
    
    # output
    df_ask = new_vols_ask[['ask','new_vol','ask_added_vol']]
    df_ask.rename(columns={'new_vol':'ask_size'}, inplace=True)
    df_out = pd.concat([df_bid, df_ask], axis=1)

    # fill bid limit orders
    bids_to_fill = new_vols_bid[(new_vols_bid['original_vol']!=0) & (new_vols_bid['next_vol']==0)][['bid_added_vol','bid']]

    for vol_to_fill in bids_to_fill['bid_added_vol'].values:

        # Hedge transaction in origin market (Kraken)
        next_ob_krak_asks['accum_size'] = next_ob_krak_asks['ask_size'].cumsum()
        next_ob_krak_asks['remaining_vol'] = vol_to_fill - next_ob_krak_asks['accum_size']
        to_drop = next_ob_krak_asks[next_ob_krak_asks['remaining_vol']>0]

        next_ob_krak_asks.drop(to_drop.index, inplace=True)
        next_ob_krak_asks.reset_index(drop=True, inplace=True)

        new_vol = -next_ob_krak_asks.iloc[0,-1] # remaing volume on surviving level

        next_ob_krak_asks.iloc[0,1] = new_vol
        next_ob_krak_asks.drop(['accum_size','remaining_vol'], axis=1, inplace=True)
        next_ob_krak_asks.reset_index(drop=True, inplace=True)



    # fill ask limit orders
    asks_to_fill = new_vols_ask[(new_vols_ask['original_vol']!=0) & (new_vols_ask['next_vol']==0)][['ask_added_vol','ask']]

    for vol_to_fill in asks_to_fill['ask_added_vol'].values:

        # Hedge transaction in origin market (Kraken)
        next_ob_krak_bids['accum_size'] = next_ob_krak_bids['bid_size'].cumsum()
        next_ob_krak_bids['remaining_vol'] = vol_to_fill - next_ob_krak_bids['accum_size']
        to_drop = next_ob_krak_bids[next_ob_krak_bids['remaining_vol']>0]

        next_ob_krak_bids.drop(to_drop.index, inplace=True)
        next_ob_krak_bids.reset_index(drop=True, inplace=True)

        new_vol = -next_ob_krak_bids.iloc[0,-1] # remaing volume on surviving level

        next_ob_krak_bids.iloc[0,1] = new_vol
        next_ob_krak_bids.drop(['accum_size','remaining_vol'], axis=1, inplace=True)
        next_ob_krak_bids.reset_index(drop=True, inplace=True)

                
    df_bit = df_out
    bit_ask = df_bit[['ask','ask_size','ask_added_vol']]
    bit_bid = df_bit[['bid_added_vol','bid_size','bid']]

    df_krak = pd.concat([next_ob_krak_bids, next_ob_krak_asks], axis=1)        
    mid_krak = (df_krak['bid'][0]+df_krak['ask'][0])/2
    ob_xemm[list(ob_bit.keys())[t]] = df_out

<ipython-input-11-02e4c08c23cd>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bit_ask['ask_added_vol'] = np.zeros(bit_ask.shape[0])
<ipython-input-11-02e4c08c23cd>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bit_bid['bid_added_vol'] = np.zeros(bit_bid.shape[0])
C:\Users\if724\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [192]:
bit_ob = pd.concat([bit_bid, bit_ask], axis=1)

In [25]:
new_tob = (ob_bit[list(ob_bit.keys())[1]].loc['0','bid'], ob_bit[list(ob_bit.keys())[1]].loc['0','ask'])

In [27]:
bids_to_drop = bit_ob[bit_ob['bid']>new_tob[0]]
asks_to_drop = bit_ob[bit_ob['ask']<new_tob[1]]

In [28]:
bit_ob.drop(bids_to_drop.index, inplace=True)
bit_ob.reset_index(drop=True, inplace=True)

In [50]:
next_ob_bit = ob_bit[list(ob_bit.keys())[1]]

In [145]:
# OrderBook resultante de XEMM (destination)
bit_bid_new_tob = bit_ob[['bid_added_vol','bid_size','bid']]
bit_ask_new_tob = bit_ob[['ask_added_vol','ask_size','ask']]

# Orderbook original siguiente timestamp (destination)
next_ob_bit_bids = next_ob_bit[['bid_size','bid']]
next_ob_bit_asks = next_ob_bit[['ask_size','ask']]

# Orderbook original siguiente timestamp (origin)
next_ob_krak = ob_krak[list(ob_krak.keys())[1]] #'i+1'
next_ob_krak_bids = next_ob_krak[['bid_size','bid']]
next_ob_krak_asks = next_ob_krak[['ask_size','ask']]

In [171]:
# generate new OrderBook based on next OB data

# Bids

# data prep
new_vols_bid = pd.merge(bit_bid_new_tob, next_ob_bit_bids, on='bid', how='outer').fillna(0)
new_vols_bid.sort_values(by='bid', ascending=False, ignore_index=True, inplace=True)
new_vols_bid.rename(columns={'bid_size_x':'current_vol','bid_size_y':'next_vol'}, inplace=True)

# procedure
new_vols_bid['original_vol'] = new_vols_bid['current_vol'] - new_vols_bid['bid_added_vol']

# scenario a: existing level w/o volume in next ob
new_vols_bid['new_vol'] = np.zeros(new_vols_bid.shape[0]) 

# scenario b: added level with volume in next ob
indx_b = new_vols_bid[(new_vols_bid['original_vol']==0) & (new_vols_bid['next_vol']!=0)].index
new_vols_bid.loc[indx_b,'new_vol'] = new_vols_bid.loc[indx_b, 'current_vol'] + new_vols_bid.loc[indx_b,'next_vol'] 

# scenario c: added level w/o volume in next ob
indx_c = new_vols_bid[(new_vols_bid['original_vol']==0) & (new_vols_bid['next_vol']==0)].index
new_vols_bid.loc[indx_c,'new_vol'] = new_vols_bid.loc[indx_c, 'bid_added_vol']

# scenario d: existing level with volume in next ob

indx_d = new_vols_bid[(new_vols_bid['original_vol']!=0) & (new_vols_bid['next_vol']!=0)].index
new_vols_bid.loc[indx_d,'new_vol'] = new_vols_bid.loc[indx_d, 'current_vol'] + (new_vols_bid.loc[indx_d, 'next_vol'] 
                                                                                - new_vols_bid.loc[indx_d, 'original_vol'])

# identify filled orders

# output
df_bid = new_vols_bid[['new_vol', 'bid']]
df_bid.rename(columns={'new_vol':'bid_size'}, inplace=True)

# Asks

# data prep
new_vols_ask = pd.merge(bit_ask_new_tob, next_ob_bit_asks, on='ask', how='outer').fillna(0)
new_vols_ask.sort_values(by='ask', ignore_index=True, inplace=True)
new_vols_ask.rename(columns={'ask_size_x':'current_vol','ask_size_y':'next_vol'}, inplace=True)

# procedure
new_vols_ask['original_vol'] = new_vols_ask['current_vol'] - new_vols_ask['ask_added_vol']
new_vols_ask.drop(new_vols_ask[new_vols_ask['ask']==0].index, inplace=True) # drop levels whose price is 0
new_vols_ask.reset_index(drop=True, inplace=True)

# scenario a: existing level w/o volume in next ob
new_vols_ask['new_vol'] = np.zeros(new_vols_ask.shape[0]) 

# scenario b: added level with volume in next ob
indx_b = new_vols_ask[(new_vols_ask['original_vol']==0) & (new_vols_ask['next_vol']!=0)].index
new_vols_ask.loc[indx_b,'new_vol'] = new_vols_ask.loc[indx_b, 'current_vol'] + new_vols_ask.loc[indx_b,'next_vol'] 

# scenario c: added level w/o volume in next ob
indx_c = new_vols_ask[(new_vols_ask['original_vol']==0) & (new_vols_ask['next_vol']==0)].index
new_vols_ask.loc[indx_c,'new_vol'] = new_vols_ask.loc[indx_c, 'ask_added_vol']

# scenario d: existing level with volume in next ob
indx_d = new_vols_ask[(new_vols_ask['original_vol']!=0) & (new_vols_ask['next_vol']!=0)].index
new_vols_ask.loc[indx_d,'new_vol'] = new_vols_ask.loc[indx_d, 'current_vol'] + (new_vols_ask.loc[indx_d, 'next_vol'] 
                                                                                - new_vols_ask.loc[indx_d, 'original_vol'])

# output
df_ask = new_vols_ask[['new_vol', 'ask']]
df_ask.rename(columns={'new_vol':'ask_size'}, inplace=True)
df_out = pd.concat([df_bid, df_ask], axis=1)



C:\Users\if724\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\if724\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [181]:
df_out

,bid_size,bid,ask_size,ask
0,14.605659,28275.0,0.011063,28278.0
1,0.001000,28274.7,0.006000,28279.0
2,1.732000,28274.3,0.100000,28280.0
3,0.100000,28273.4,0.050000,28281.0
4,0.005124,28273.0,0.323781,28282.0
5,1.490000,28272.6,1.580312,28283.0
6,0.010158,28272.0,0.353892,28284.0
7,0.002000,28270.3,1.053899,28285.0
8,0.003000,28268.0,0.950994,28286.0
9,1.581528,28266.0,0.157200,28287.0


In [174]:
# fill bid limit orders
bids_to_fill = new_vols_bid[(new_vols_bid['original_vol']!=0) & (new_vols_bid['next_vol']==0)][['bid_added_vol','bid']]

for vol_to_fill in bids_to_fill['bid_added_vol'].values:
    
    # Hedge transaction in origin market (Kraken)
    next_ob_krak_asks['accum_size'] = next_ob_krak_asks['ask_size'].cumsum()
    next_ob_krak_asks['remaining_vol'] = vol_to_fill - next_ob_krak_asks['accum_size']
    to_drop = next_ob_krak_asks[next_ob_krak_asks['remaining_vol']>0]
    
    next_ob_krak_asks.drop(to_drop.index, inplace=True)
    next_ob_krak_asks.reset_index(drop=True, inplace=True)
    
    new_vol = -next_ob_krak_asks.iloc[0,-1] # remaing volume on surviving level
    
    next_ob_krak_asks.iloc[0,1] = new_vol
    next_ob_krak_asks.drop(['accum_size','remaining_vol'], axis=1, inplace=True)
    next_ob_krak_asks.reset_index(drop=True, inplace=True)
    

    
# fill ask limit orders
asks_to_fill = new_vols_ask[(new_vols_ask['original_vol']!=0) & (new_vols_ask['next_vol']==0)][['ask_added_vol','ask']]

for vol_to_fill in asks_to_fill['ask_added_vol'].values:
    
    # Hedge transaction in origin market (Kraken)
    next_ob_krak_bids['accum_size'] = next_ob_krak_bids['bid_size'].cumsum()
    next_ob_krak_bids['remaining_vol'] = vol_to_fill - next_ob_krak_bids['accum_size']
    to_drop = next_ob_krak_bids[next_ob_krak_bids['remaining_vol']>0]
    
    next_ob_krak_bids.drop(to_drop.index, inplace=True)
    next_ob_krak_bids.reset_index(drop=True, inplace=True)
    
    new_vol = -next_ob_krak_bids.iloc[0,-1] # remaing volume on surviving level
    
    next_ob_krak_bids.iloc[0,1] = new_vol
    next_ob_krak_bids.drop(['accum_size','remaining_vol'], axis=1, inplace=True)
    next_ob_krak_bids.reset_index(drop=True, inplace=True)
    

### lskdsal;jd


# Next steps

1. Sacar diferencias en volumen de cada nivel entre el OB output `ob_bit` y el siguiente OB destination.
**Termina ciclo**